In [ ]:
import tqdm, re, pandas as pd, random, pprint, json
import matplotlib.pyplot as plt
import pickle as pkl
from wordcloud import WordCloud, STOPWORDS
stopwords= set(STOPWORDS)
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary

In [ ]:
def get_memory():
    status= open('/proc/self/status').read()
    memuse= int(re.findall('VmRSS:\s*(\d+)\skB', status)[0]) / 1024 ** 2
    print('Resident memory: %.2f GB' % memuse)

In [ ]:
get_memory()

In [ ]:
def loadPickle(name):
    return pd.read_pickle(f'../data/{name}')
def loadTxt(name):
    return [i.strip() for i in open(f'../data/{name}', 'r').readlines()]
def dumpPickle(df, name):
    df.to_pickle(f'../data/{name}')

In [ ]:
def showTopics(data):
    # create dictionary
    splitDf= data.map(lambda x: [i.lower() for i in re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x).split() if len(i)>=3 and i not in stopwords])
    text_dict= Dictionary(splitDf)
    # Output the frequent topics LDA Model
    tweets_bow= [text_dict.doc2bow(tweet) for tweet in splitDf]
    tweets_lda= LdaModel(tweets_bow, num_topics= 1, id2word= text_dict, random_state= 1, passes= 10)
    for idx, topic in tweets_lda.show_topics(formatted=False):
        print('Topic: {} \nWords: {}'.format(idx, [w[0] for w in topic]))

In [ ]:
suspendedLang= json.load(open('../../data/suspended_tweets_lang_accuracy.json',))
restoredLang= json.load(open('../../data/restored_tweets_lang_accuracy.json',))
langDict= suspendedLang
for i in restoredLang:
    langDict[i]= restoredLang[i]

In [ ]:
temp= pd.read_csv('./exported-nodes-network-usermentions-suspended-restored.csv')
print(temp[temp['Cluster']<=4]['Cluster'].value_counts())
mentionedUsers= {}
for i, r in temp[temp['Cluster']<=4][['Cluster', 'Id']].iterrows():
    mentionedUsers[str(r['Id'])]= r['Cluster']
temp= pd.read_csv('./exported-nodes-network-suspended-restored.csv')
print(temp[temp['Cluster']<=4]['Cluster'].value_counts())
retweetedUsers= {}
for i, r in temp[temp['Cluster']<=4][['Cluster', 'Id']].iterrows():
    retweetedUsers[str(r['Id'])]= r['Cluster']

In [ ]:
userMentions= [{'full_text': []}, {'full_text': []}, {'full_text': []}, {'full_text': []}, {'full_text': []}]
with open(f'../../data/all_tweets_suspended_jan3_2021.pickle', 'rb') as f:
    try:
        while True:
            temp= pkl.load(f)
            for i in tqdm.tqdm(temp):
                if i.get('retweeted_status') is None and langDict[i['id_str']]['lang']=='en':
                    if mentionedUsers.get(i['user']['id_str']) is not None:
                        clusterIndex= mentionedUsers[i['user']['id_str']]
                        if i.get('text'):
                            userMentions[clusterIndex]['full_text'].append(i['text'])
                        elif i.get('full_text'):
                            userMentions[clusterIndex]['full_text'].append(i['full_text'])
    except EOFError:
        pass

In [ ]:
for i in userMentions:
    showTopics(pd.DataFrame.from_dict(i)['full_text'])
    print('\n')

In [ ]:
userRetweets= [{'full_text': []}, {'full_text': []}, {'full_text': []}, {'full_text': []}, {'full_text': []}]
with open(f'../../data/all_tweets_suspended_jan3_2021.pickle', 'rb') as f:
    try:
        while True:
            temp= pkl.load(f)
            for i in tqdm.tqdm(temp):
                if i.get('retweeted_status') is None and langDict[i['id_str']]['lang']=='en':
                    if retweetedUsers.get(i['user']['id_str']) is not None:
                        clusterIndex= retweetedUsers[i['user']['id_str']]
                        if i.get('text'):
                            userRetweets[clusterIndex]['full_text'].append(i['text'])
                        elif i.get('full_text'):
                            userRetweets[clusterIndex]['full_text'].append(i['full_text'])
    except EOFError:
        pass

In [ ]:
for i in userRetweets:
    showTopics(pd.DataFrame.from_dict(i)['full_text'])
    print('\n')